In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, StandardScaler

data = {
    'Meal': ['Breakfast', 'Lunch', 'Dinner', 'Snack'],
    'Nutrition Goals': ['Lose Weight', 'Gain Muscle', 'Lose Weight', 'Maintain Weight'],
    'Workout Intensity': ['High', 'Medium', 'High', 'Low'],
    'Recipe': ['Omelette', 'Grilled Chicken Salad', 'Salmon with Veggies', 'Greek Yogurt with Berries'],
    'Pantry': ['Eggs, Spinach', 'Chicken, Lettuce', 'Salmon, Broccoli', 'Greek Yogurt, Berries'],
}

df = pd.DataFrame(data)

encoder = LabelEncoder()
df['Meal_ID'] = encoder.fit_transform(df['Meal'])
df['Nutrition_Goals_ID'] = encoder.fit_transform(df['Nutrition Goals'])
df['Workout_Intensity_ID'] = encoder.fit_transform(df['Workout Intensity'])

pantry_items = df['Pantry'].str.get_dummies(', ')
pantry_items.columns = [f'Pantry_{item}' for item in pantry_items.columns]

features = pd.concat([df[['Meal_ID', 'Nutrition_Goals_ID', 'Workout_Intensity_ID']], pantry_items], axis=1)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

input_layer = Input(shape=(scaled_features.shape[1],), name='Input')

dense_layer_1 = Dense(16, activation='relu')(input_layer)
dense_layer_2 = Dense(8, activation='relu')(dense_layer_1)
output_layer = Dense(len(df), activation='softmax')(dense_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

X = scaled_features
y = tf.keras.utils.to_categorical(df.index)

model.fit(X, y, epochs=50, batch_size=4, verbose=0)

alex_data = {
    'Meal_ID': [0],  
    'Nutrition_Goals_ID': [0], 
    'Workout_Intensity_ID': [2], 
    'Pantry_Eggs': [1],
    'Pantry_Spinach': [1],
    'Pantry_Chicken': [0],
    'Pantry_Lettuce': [0],
    'Pantry_Salmon': [0],
    'Pantry_Broccoli': [0],
    'Pantry_Greek Yogurt': [1],
    'Pantry_Berries': [1],
}

alex_features = pd.DataFrame(alex_data)
alex_scaled_features = scaler.transform(alex_features[features.columns])  # Ensure feature order matches
alex_recommendations = model.predict(alex_scaled_features)

top_recommendations = alex_recommendations.argsort()[0][-3:][::-1]
recommended_recipes = df['Recipe'].iloc[top_recommendations].tolist()

print("Recipe recommendations for Alex:")
for recipe in recommended_recipes:
    print(recipe)


1/1 [==============================] - 0s 84ms/step
Recipe recommendations for Alex:
Greek Yogurt with Berries
Omelette
Grilled Chicken Salad


In [8]:
In MyFitnessPal, MLNE helps Alex log her meals and recommends dietary adjustments based
on her nutrition goals and workout intensity. It could suggest new recipes that align with her dietary goals
and what she has in her pantry, optimizing her meal planning process.

SyntaxError: ignored